In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np
import pandas as pd

In [ ]:
class LogicGateModel:
    '''This class models a two-input logic gate function, i.e., AND, OR, XOR, NAND, NOR, XNOR
    hidden_layers gives a list of layer specifications of the form: (#ofNodes, activationFunction);
    output_layer is similar but with only layer specification.
    '''

    def __init__(self,
                 optimizer=SGD(lr=0.1),
                 hidden_layers=[],
                 output_layer=(1, tf.keras.activations.linear),
                 loss_function=tf.keras.losses.mse
                 ):
        self.model = Sequential()
        for layer in hidden_layers:
            self.model.add(Dense(layer[0], input_dim=2, activation=layer[1]))
        self.model.add(Dense(output_layer[0], input_dim=2, activation=output_layer[1]))
        self.model.compile(loss=loss_function,
                           optimizer=optimizer,
                           metrics=[tf.keras.metrics.binary_accuracy]
                           )

    def train(self, X, y, n_epochs=100):
        self.model.fit(X, y, batch_size=1, epochs=n_epochs, verbose=0)

    def predict(self, X):
        return self.model.predict(X)

In [4]:
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y_and = np.array([[0],[0],[0],[1]])
y_or = np.array([[0],[1],[1],[1]])
y_xor = np.array([[0],[1],[1],[0]])

# Print the correct answers.
print('Correct answers')
print("         AND OR XOR")
for i in range(4):
    print('{} ->  '.format(X[i]),
          '{}  '.format(X[i,0] & X[i,1]),
          '{}   '.format(X[i,0] | X[i,1]),
          '{}'.format(X[i,0] ^ X[i,1])
          )

Correct answers
         AND OR XOR
[0 0] ->   0   0    0
[0 1] ->   0   1    1
[1 0] ->   0   1    1
[1 1] ->   1   1    0


In [10]:
model = LogicGateModel(
    hidden_layers=[(2, tf.nn.tanh)],
    output_layer=(1, tf.nn.sigmoid),
    loss_function=tf.keras.losses.binary_crossentropy
)
model.train(X, y_xor, n_epochs=1000)
print(model.predict(X))

[[0.00682185]
 [0.98812985]
 [0.98812675]
 [0.00577613]]


I reduced the dimensionality of the hidden layer to 2, as this did not cause much loss in the accuracy of the output.

# Part 2

In [12]:
boston_df = pd.read_csv("https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv")
boston_df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [20]:
print(len(boston_df), 'total data points')
print('across', len(boston_df.keys()), 'differnet columns')


506 total data points
across 14 differnet columns


In [24]:
# Split the boston DF into 3 parts
def split_data(df):
    train = df.head(400)
    val = df[400:450]
    test = df[450:]
    
    return (train, val, test)


In [ ]:
boston_df['tax_per_value'] = boston_df['tax'] / boston_df['medv']
print(boston_df['tax_per_value'])

This adds the tax amount per house value. This could be useful for making inferences about which areas have higher property tax without data being skewed by variations in house value.